In [ ]:
# Manual annotation of positive / negative markers
# using napari
# Saves annotations for each celtype as napari point label .csv (x,y positions)
# authors: Pacome Prompsy
# contact: pacome.prompsy@chuv.ch
# Guenova Lab
# CHUV (Centre Hospitalier Universitaire Vaudois), Lausanne, Suisse


In [ ]:
# this cell is required to run these notebooks on Binder
# if running on Binder, remember to **WAIT 5 SECONDS** after
# running the '%gui qt' cell below. If you get an error,
# click on "Kernel -> Restart" and try again. Make sure also
# that you have a desktop tab open.
import os
#if 'BINDER_SERVICE_HOST' in os.environ:
#    os.environ['DISPLAY'] = ':1.0'

In [ ]:
os.environ["QT_QPA_PLATFORM"] = "wayland"

In [ ]:
##############################################################################################################
##############################################################################################################


##############################################################################################################
##############################################################################################################

In [ ]:
import os
import pandas as pd
import numpy as np
from tifffile import imread
import random
import re

%gui qt

In [ ]:
import napari
from napari.utils import nbscreenshot

# Create an empty viewer
viewer = napari.Viewer()

In [ ]:
# Change below
output_dir = "../output/manual_annotation_marker2/"
tiff_dir = "../output/input/"
segmentation_dir  = "../output/segmentation/"
marker_file  = "../annotation/marker_metadata.csv"


In [ ]:
marker_df = pd.read_csv(os.path.join(marker_file))
marker_unique = marker_df.Marker[(marker_df.Marker != "DAPI") & (marker_df.PassOverallQuality == True)]
marker_unique = marker_unique.values


In [ ]:
# CHANGE HERE SAMPLE :
samples = ["ROI-30","ROI-40","ROI-50","ROI-60"]
sample = samples[0]
markers = ["Cytokeratin"]
marker = markers[0]
name = "Annotator1"
range_sample = -1


In [ ]:
@viewer.bind_key('Ctrl-Shift-L', overwrite = "True")
def visible_1(viewer):
    
    print("Saving...")
    global sample
    global range_sample

    save_dir = os.path.join(output_dir, sample + "-" + name )FFFF
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    if marker + "+" in viewer.layers:
        viewer.layers[marker + "+"].save(os.path.join( save_dir, marker + "+" + '.csv'));
        viewer.layers.remove(value = marker + "+")
    if marker + "-" in viewer.layers:
        viewer.layers[marker + "-"].save(os.path.join( save_dir, marker + "-" + '.csv'));
        viewer.layers.remove(value = marker + "-")
    if sample + "-" + marker in viewer.layers:
        viewer.layers.remove(value = sample + "-" + marker)
        
    range_sample = range_sample + 1
    sample = samples[range_sample]
    
    DAPI = imread(os.path.join(os.path.join(tiff_dir, sample, "DAPI.tiff")))
    viewer.add_image(DAPI, name="DAPI", colormap="bop blue",contrast_limits=[5000, 65000]);

    if sample + "-" + marker not in viewer.layers:

        image = imread(os.path.join(os.path.join(tiff_dir, sample, marker + ".tiff")))
        
        if sample in ["ROI-01", "ROI-10", "ROI-20"]:
            contrast_limits=[marker_df.ContrastRange_min[marker_df.Marker == marker].values[0],
                                          marker_df.ContrastRange_max[marker_df.Marker == marker].values[0]]
        else:
            contrast_limits=[marker_df.ContrastRange_min_20230601[marker_df.Marker == marker].values[0],
                                          marker_df.ContrastRange_max_20230601[marker_df.Marker == marker].values[0]]
        
        viewer.add_image(image, name=sample + "-" + marker, blending= "additive",
                         contrast_limits=contrast_limits);
        viewer.add_points(name= marker + "-", face_color = "red",
                      edge_color = "#d4c51bff", edge_width=0.1, opacity=1, size = 30);
        viewer.layers[marker + "-"].mode = "add"
        viewer.add_points(name= marker + "+", face_color = "green",
                      edge_color = "#d4c51bff", edge_width=0.1, opacity=1, size = 30);
        viewer.layers[marker + "+"].mode = "add"


    if "cell_types" in viewer.layers:
        viewer.layers.remove(value="cell_types")

    segmentation = imread(os.path.join(os.path.join(segmentation_dir, sample + "_whole_cell_segmentation_borders.tiff")))
    viewer.add_image(segmentation, name="cell_types", colormap = "red", blending = "additive");



In [ ]:
marker_df.ContrastRange_max[marker_df.Marker == marker].values[0]

In [ ]:
@viewer.bind_key('Shift-S', overwrite = "True")
def save_all(viewer):
    save_dir = os.path.join(output_dir, sample + "-" + name )
    print("Saving...")
    for marker in markers:
        if marker + "+" in viewer.layers:
            viewer.layers[marker + "+"].save(os.path.join( save_dir, marker + "+" + '.csv'));
        if marker + "-" in viewer.layers:
            viewer.layers[marker + "-"].save(os.path.join( save_dir, marker + "-" + '.csv'));

In [ ]:
@viewer.bind_key('Shift->', overwrite = "True")
def down_contrast_start(viewer):
    point = viewer.layers.selection.active
    marker = re.sub("-|\+", "", point.name)
    if marker in viewer.layers:
        increment = 50 + round(0.05 * viewer.layers[marker].contrast_limits[0])
        viewer.layers[marker].contrast_limits = [max(0,  viewer.layers[marker].contrast_limits[0] - increment),
                                                     viewer.layers[marker].contrast_limits[1]]

In [ ]:
@viewer.bind_key('Shift-Y', overwrite = "True")
def down_contrast_start(viewer):
    point = viewer.layers.selection.active
    marker = re.sub("-|\+", "", point.name)
    if marker in viewer.layers:
        increment = 50 + round(0.05 * viewer.layers[marker].contrast_limits[0])
        viewer.layers[marker].contrast_limits = [viewer.layers[marker].contrast_limits[0],
                                                     max(viewer.layers[marker].contrast_limits[0] + 10,  viewer.layers[marker].contrast_limits[1]  - increment)]

In [ ]:
@viewer.bind_key('Ctrl-Shift->', overwrite = "True")
def down_contrast_start(viewer):
    point = viewer.layers.selection.active
    marker = re.sub("-|\+", "", point.name)
    if marker in viewer.layers:
        increment = 50 + round(0.05 * viewer.layers[marker].contrast_limits[0])
        viewer.layers[marker].contrast_limits = [min(viewer.layers[marker].contrast_limits[1]-10,  viewer.layers[marker].contrast_limits[0] + increment),
                                                     viewer.layers[marker].contrast_limits[1]]

In [ ]:
@viewer.bind_key('Ctrl-Shift-Y', overwrite = "True")
def down_contrast_start(viewer):
    point = viewer.layers.selection.active
    print(point.name)
    marker = re.sub("-|\+", "", point.name)
    print(marker)
    if marker in viewer.layers:
        increment = 50 + round(0.05 * viewer.layers[marker].contrast_limits[0])
        viewer.layers[marker].contrast_limits = [viewer.layers[marker].contrast_limits[0],
                                                     min(65000,  viewer.layers[marker].contrast_limits[1]  + increment)]

In [ ]:
@viewer.bind_key('Shift-D', overwrite = "True")
def visible_1(viewer):
    viewer.layers["DAPI"].visible = not viewer.layers["DAPI"].visible

In [ ]:
@viewer.bind_key('Shift-X', overwrite = "True")
def unvisible_all(viewer):
    for layer in viewer.layers:
        if (not "-" in layer.name) and (not "+" in layer.name) and (layer.name != "background")  and (layer.name != "cell_types"):
            layer.visible = False

In [ ]:
@viewer.bind_key('Shift-F', overwrite = "True")
def visible_1(viewer):
    print(sample)
    viewer.layers[sample + "-" + marker].visible = not viewer.layers[sample + "-" + marker].visible